# Demo: Defining Control_M Workflows using Python

# Step 1 - Setup

## Step 1A - Install the library

In [1]:
!pip --version

pip 21.2.4 from /opt/conda/lib/python3.8/site-packages/pip (python 3.8)


In [2]:
!pip install --upgrade --no-deps --force-reinstall git+https://github.com/tadinve/naga.git

  Cloning https://github.com/tadinve/naga.git to /tmp/pip-req-build-0mbq1ng_
  Running command git clone -q https://github.com/tadinve/naga.git /tmp/pip-req-build-0mbq1ng_
  Resolved https://github.com/tadinve/naga.git to commit a49f55381af4ddbb0b01eda282146e815dc76fda
  Created wheel for ctm-python-client: filename=ctm_python_client-0.3.0-py3-none-any.whl size=615430 sha256=6501989aa660853c3ccfc227e3432aea22c2145517cddd10d457032c4306a0e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-57f996y_/wheels/de/35/cb/b70c2ffd6e40942c62635105854bc5fabd5125a0f0f9b17cb9
Successfully built ctm-python-client
  Attempting uninstall: ctm-python-client
    Found existing installation: ctm-python-client 0.3.0
    Uninstalling ctm-python-client-0.3.0:
      Successfully uninstalled ctm-python-client-0.3.0


In [3]:
from ctm_python_client.core.bmc_control_m import CmJobFlow
from ctm_python_client.jobs.dummy import DummyJob

# Step 2 - Instantiate, Authenticate and Schedule


## Step 2A - Create the object

In [4]:
# Please change the URfrI, and ctm_user and enter ctm_password to match your environment
from ctm_python_client.session.session import Session

import getpass
ctm_uri =  "https://acb-rhctmv20.centralus.cloudapp.azure.com:8443/automation-api"
ctm_user =  "vtadinad"
ctm_pwd =                                                                                                                                               "P4ssw0rd"
if "ctm_pwd" not in locals(): # has not been enterd once, will skip next time
    ctm_pwd = getpass.getpass("Enter your Control M Password ")

session = Session( endpoint=ctm_uri,username=ctm_user,password=ctm_pwd)
session.get_token()

'46FAE00ED1EB392DDDF11A86EB7DE625E3DD48568ED23C0842716C0E5EF4CCABACA27A739B2E57C6BFF353B34F877D91F57F072A5B4FA7CE67D8B173B9669CB8'

In [5]:
t1_flow = CmJobFlow(application="Naga0.3_Examples",sub_application="Demo-OR_JOB",session=session)

## Step 2B - Define the Schedule

In [6]:
t1_flow.set_run_as(username="ctmuser",host="acb-rhctmv20")

In [7]:
# Define the schedule
months = ["JAN", "OCT", "DEC"]
monthDays = ["ALL"]
weekDays = ["MON","TUE", "WED", "THU", "FRI"]
fromTime = "0300"
toTime = "2100"
t1_flow.set_schedule(months, monthDays, weekDays,fromTime, toTime)


# Step 3  - Create Folder

In [8]:
# Create Fodler
f1 = t1_flow.create_folder(name="OR-JOB")

# Step 4 - Create Tasks

In [9]:
start = t1_flow.add_job( f1, DummyJob(f1,"Start-Flow") )


job1 = DummyJob(f1,"Job1")
job1.add_if_output("if-true", "*true*", "Job1-TO-Job2")
job1.add_if_output("if-flase", "*false*", "Job1-TO-Job3")
job1_id = t1_flow.add_job( f1, job1 )

job2 = DummyJob(f1,"Job2")
job2_id = t1_flow.add_job( f1, job2 )

job3 = DummyJob(f1,"Job3")
job3_id = t1_flow.add_job( f1, job3 )

job4 = DummyJob(f1,"Job4")
job4.wait_for_jobs(job1_id, job3_id, condition="OR")
job4_id = t1_flow.add_job( f1, job4 )

end = t1_flow.add_job(f1, DummyJob(f1,"End-Flow") )



TypeError: append() takes exactly one argument (3 given)

# Step 5 - Chain Tasks

In [ ]:
# start >>  hello_world_id >> end
t1_flow.chain_jobs(f1,[start, job1_id])
t1_flow.chain_jobs(f1,[job2_id, job3_id])
t1_flow.chain_jobs(f1,[job4_id, end])

# Step 6 - Display Workflow

## Step 6A - Display DAG

In [ ]:
# View the t1_flow Details
nodes, edges = t1_flow.get_nodes_and_edges()
nodes, edges

In [ ]:
#display using graphviz
from ctm_python_client.utils.displayDAG import DisplayDAG

#sudo apt-get install graphviz (on unix)
#  or 
# brew install graphviz (for mac)
#DisplayDAG(t1_flow).display_graphviz()

## Step 6B - Display JSON

In [ ]:
t1_flow.display_json()

# Step 7 - Submit Workflow to Control-M

In [ ]:
t1_flow.deploy()

In [ ]:
t1_flow.run()